# 시카고 샌드위치 맛집 분석
파이썬으로데이터주무르기 책139p  
goo.gl/wAtv1s

## 1. 시카고 샌드위치 맛집 사이트에 접근하기

In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen
import pandas as pd 
from tqdm import tqdm_notebook

In [2]:
url_base = 'http://www.chicagomag.com'
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub

html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

In [3]:
rest_list = soup.select('.sammy')
rest_list[0]

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
Old Oak Tap<br>
<em>Read more</em> </br></br></a></div>
</div>

In [4]:
len(rest_list)

50

## 2. 접근한 웹 페이지에서 원하는 데이터 추출하고 정리하기

In [5]:
rank = rest_list[0].select_one('.sammyRank').string
rank

'1'

In [6]:
from urllib.parse import urljoin
url = urljoin(url_base, rest_list[10].find('a')['href'])
url

'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Lula-Cafe-Ham-and-Raclette-Panino/'

In [7]:
tmp = rest_list[0].find('a').get_text()
tmp

'BLT\r\nOld Oak Tap\nRead more '

In [8]:
tmp.split('\n')

['BLT\r', 'Old Oak Tap', 'Read more ']

In [9]:
menu = tmp.split('\n')[0].replace('\r', '')
cafe = tmp.split('\n')[1]
menu, cafe

('BLT', 'Old Oak Tap')

In [10]:
rank_list = []; url_list = []; menu_list = []; cafe_list = []
for rest in rest_list:
    rank = int(rest.select_one('.sammyRank').string)
    url = urljoin(url_base, rest.find('a')['href'])
    tmp = rest.find('a').get_text().split('\n')
    menu = tmp[0].replace('\r', '')
    cafe = tmp[1]
    rank_list.append(rank)
    url_list.append(url)
    menu_list.append(menu)
    cafe_list.append(cafe)

In [11]:
import pandas as pd 
df = pd.DataFrame({
    'Rank': rank_list,
    'Cafe': cafe_list,
    'Menu': menu_list,
    'URL': url_list
})
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...
1,2,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,3,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,4,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,5,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [12]:
#데이터프레임만들고 파일로 만들어놓는게 좋다
#다음번엔 파일열어서 작업하면 되니까~_~
df.to_csv('chicago.csv', sep=',', encoding='utf8')

## 3. 다수의 웹 페이지에 자동으로 접근해서 원하는 정보 가져오기

In [13]:
html = urlopen(df['URL'][0])
soup_tmp = BeautifulSoup(html, 'html.parser')

In [14]:
soup_tmp.select_one('.addy')

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>

In [15]:
soup_tmp.select_one('.addy').get_text()

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [16]:
tmp = soup_tmp.select_one('.addy').get_text().split(',')
len(tmp)

3

In [26]:
tmp #[0]금액 주소, [1]전번, [2]홈피

['\n$6.85. 3351 N. Broadway', ' 773-868-4000', ' phoebesbakery.com']

In [17]:
tmp[0]

'\n$10. 2109 W. Chicago Ave.'

In [18]:
price = tmp[0].split()[0][:-1]   #0번째 점삭제[:-1]
price

'$10'

In [19]:
addr = ' '.join(tmp[0].split()[1:])
addr

'2109 W. Chicago Ave.'

In [20]:
tmp[1].strip()  #전화번호 

'773-772-0406'

In [21]:
from tqdm import tqdm_notebook

In [22]:
rank_list = []; url_list = []; menu_list = []; cafe_list = []
price_list = []; addr_list = []; tel_list = []; hp_list = []
for rest in tqdm_notebook(rest_list):
    rank = int(rest.select_one('.sammyRank').string)
    url = urljoin(url_base, rest.find('a')['href'])
    tmp = rest.find('a').get_text().split('\n')
    menu = tmp[0].replace('\r', '')
    cafe = tmp[1]
    rank_list.append(rank)
    url_list.append(url)
    menu_list.append(menu)
    cafe_list.append(cafe)

    html = urlopen(url)
    soup_tmp = BeautifulSoup(html, 'html.parser')
    tmp = soup_tmp.select_one('.addy').get_text().split(',')
    price = tmp[0].split()[0][:-1]
    price_list.append(price)
    addr = ' '.join(tmp[0].split()[1:])
    if len(tmp) == 1:
        addr_list.append(addr)
        tel_list.append(' ')
        hp_list.append(' ')
    elif len(tmp) == 2:
        if addr.find('Multiple') >= 0:
            addr_list.append(' ')
            tel_list.append(' ')
            hp_list.append(tmp[1])
        else:
            addr_list.append(addr)
            tel_list.append(tmp[1].strip())
            hp_list.append(' ')
    elif len(tmp) == 3:
        addr_list.append(addr)
        tel_list.append(tmp[1].strip())
        hp_list.append(tmp[2])
    elif len(tmp) == 4:
        addr += ', ' + tmp[1]
        addr_list.append(addr)
        tel_list.append(tmp[2].strip())
        hp_list.append(tmp[3])

<ipython-input-22-4d76cd5bb101>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for rest in tqdm_notebook(rest_list):


In [23]:
df2 = pd.DataFrame({
    'Rank': rank_list,
    'Cafe': cafe_list,
    'Menu': menu_list,
    'Price': price_list,
    'Address': addr_list,
    'Telephone': tel_list,
    'Home Page': hp_list
})
df2.head()

,Rank,Cafe,Menu,Price,Address,Telephone,Home Page
0,1,Old Oak Tap,BLT,$10,2109 W. Chicago Ave.,773-772-0406,theoldoaktap.com
1,2,Au Cheval,Fried Bologna,$9,800 W. Randolph St.,312-929-4580,aucheval.tumblr.com
2,3,Xoco,Woodland Mushroom,$9.50,445 N. Clark St.,312-334-3688,rickbayless.com
3,4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston",847-475-9400,alsdeli.net
4,5,Publican Quality Meats,PB&L,$10,825 W. Fulton Mkt.,312-445-8977,publicanqualitymeats.com


In [24]:
df2.to_csv('chicago2.csv', sep=',', encoding='utf8')

## 3. 맛집 위치를 지도에 표기하기
google maps api key 구글링   
conda install -c conda-forge googlemaps 설치하기    
googlemapskey.txt 에 api만들어 넣고  ...........결제계정카드비번5회실패.........  
깃이그노어에 명칭추가    

In [25]:
import numpy as np
import folium
import googlemaps

ModuleNotFoundError: No module named 'folium'

In [ ]:
key_fd = open('googlemapskey.txt', mode='r')
gmaps_key = key_fd.read(100)
key_fd.close()

In [ ]:
gmaps = googlemaps.Client(key=gmaps_key)

In [ ]:
lat = []
lng = []

for n in tqdm_notebook(df2.index):
    if df2['Address'][n].find(' ') != 0:
        target_name = df2['Address'][n]+', '+'Chicago'
        gmaps_output = gmaps.geocode(target_name)
        location_output = gmaps_output[0].get('geometry')
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lng'])
    else:
        lat.append(np.nan)
        lng.append(np.nan)

In [ ]:
print(lat)

In [ ]:
df2['Lattitude'] = lat
df2['Longitude'] = lng
df2.head()

In [ ]:
mapping = folium.Map(location=[df2['Lattitude'].mean(), 
                            df2['Longitude'].mean()], zoom_start=11)
folium.Marker([df2['Lattitude'].mean(), df2['Longitude'].mean()], 
                                      popup='center').add_to(mapping)
mapping

In [ ]:
mapping = folium.Map(location=[df2['Lattitude'].mean(), 
                            df2['Longitude'].mean()], zoom_start=11)

for n in df2.index:
    if df2['Address'][n].find(' ') != 0:
        folium.Marker([df2['Lattitude'][n], df2['Longitude'][n]], 
                                      popup=df['Cafe'][n]).add_to(mapping)

mapping

In [ ]:
df2.to_csv('chicago3.csv', sep=',', encoding='utf8')